In [ ]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 
import bk.load
import bk.signal
import bk.plot
import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [14]:
def powerMap(lfp,pos,intervals,low=5,high=10):
    #Filter in the band of interess
    filtered = bk.signal.passband(lfp,low,high)
#     plt.figure()
    plt.plot(lfp)
    plt.plot(filtered)
#     plt.title(bk.load.session + ' #' +str(channel) + ' Filtered in range '+ str(low) + ' ' + str(high))
    
    # Compute analytic signal in order to have power
    xa = scipy.signal.hilbert(filtered)
    xa = nts.Tsd(np.array(filtered.index),xa)
    x_power = scipy.stats.zscore(np.abs(xa.values)**2)
#     x_power = (np.abs(xa.values)**2)
#     g = scipy.signal.gaussian(1250,500)
#     x_power = np.convolve(x_power,g,'same') 
    x_power = nts.Tsd(np.array(filtered.index),x_power*1000)
    plt.plot(x_power)
    
    #reindex position using index of power
    pos = pos.reindex(x_power.index,method = 'nearest')
    pos['power'] = x_power.values
    powerMap = np.histogram2d(pos.restrict(intervals)['x'],pos.restrict(intervals)['y'],weights=pos.restrict(intervals)['power'],bins = [50,20])
    occupancy = np.histogram2d(pos.restrict(intervals)['x'],pos.restrict(intervals)['y'],[50,20])
        
    return powerMap[0],occupancy[0]


def plotPowerMap(powermap,occupancy):
#     plt.subplot(3,1,1)
#     plt.imshow(powermap.T)
#     plt.subplot(3,1,2)
#     plt.imshow(occupancy.T)
#     plt.subplot(3,1,3)
#     plt.subplot(3,1,1)
#     plt.imshow(powermap.T)
#     plt.subplot(3,1,2)
#     plt.imshow(occupancy.T)
#     plt.subplot(3,1,3)
    plt.imshow(np.divide(powermap.T,occupancy.T))
#     sub = plt.subplot(4,1,1)
#     plt.imshow(occupancy.T)
#     plt.subplot(4,1,2)
#     plt.imshow(a.T)
#     plt.subplot(4,1,3)
#     plt.imshow(.T/occupancy.T)
#     plt.subplot(4,1,4)
#     plt.plot(x_power.as_units('s'))
#     bk.plot.intervals(inter,'orange')

In [ ]:
def main(path,channel,low,high):
    bk.load.current_session(path)
    laps = scipy.io.loadmat(bk.load.session+'-LapType.mat')
    run = scipy.io.loadmat(bk.load.session + '-TrackRunTimes.mat')
    
    runIntervals = scipy.io.loadmat('runintervals.mat')['runintervals']
    runIntervals = nts.IntervalSet(runIntervals[:,0],runIntervals[:,1],time_units='s')

    danger_laps = laps['aplaps'][0]['run'][0]
    danger_laps = nts.IntervalSet(danger_laps[:,0],danger_laps[:,1],time_units = 's')

    safe_laps = laps['safelaps'][0]['run'][0]
    safe_laps = nts.IntervalSet(safe_laps[:,0],safe_laps[:,1],time_units = 's')

    trackruntimes = run['trackruntimes']
    trackruntimes = nts.IntervalSet(trackruntimes[:,0],trackruntimes[:,1],time_units = 's')
    
    laps = {'run':trackruntimes,'danger':danger_laps,'safe':safe_laps}
    
    
    lfp = pd.Series(dtype = 'object')
    print(runIntervals.as_units('s'))
    for run in runIntervals.as_units('s').iloc:
        lfp_ = bk.load.lfp(run['start'],run['end'],166,channel)
        lfp = lfp.append(lfp_)
    
    lfp = nts.Tsd(lfp[~lfp.index.duplicated(keep='first')])
    pos = bk.load.pos()
    
    powermap,occupancy = powerMap(lfp,pos,runIntervals,low,high)
#     plt.figure()
#     plotPowerMap(powermap,occupancy)
    plt.title(bk.load.session + ' #' +str(channel) + ' Filtered in range '+ str(low) + ' ' + str(high))
    return(powermap,occupancy)

In [ ]:
path = "Z:\Rat08\Rat08-20130713"
# bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713'

channels = [23,105,150]
low = [5,50]
high = [10,80]
p = 1 
for i,c in enumerate(channels,0):
    for j,b in enumerate(zip(low,high),0):
        plt.subplot(len(channels),len(low),p)
        occupancy = main(path,c,b[0],b[1])
        p += 1
plt.figure()
plt.imshow(occupancy.T)
plt.title(path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
     start      end
0      0.0    206.3
1   9405.5  12649.0
2  22415.0  22680.0
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp


In [15]:
path = "Z:\Rat08\Rat08-20130713"
p = plt.subplot(2,1,1)
powermap,occupancy = main(path,13,5,10)
p = plt.subplot(2,1,2,sharex=p)
powermap2,occupanc2 = main(path,105,5,10)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
     start      end
0      0.0    206.3
1   9405.5  12649.0
2  22415.0  22680.0
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
     start      end
0      0.0    206.3
1   9405.5  12649.0
2  22415.0  22680.0
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp


In [43]:
plt.figure()
a = plt.subplot(2,1,1)
plt.colorbar()

plt.subplot(2,1,2,sharex = a)
plt.colorbar()
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [20]:
bk.load.current_session(path)

runIntervals = scipy.io.loadmat('runintervals.mat')['runintervals']
runIntervals = nts.IntervalSet(runIntervals[:,0],runIntervals[:,1],time_units='s')

channels = [23,105]
lfps = dict()
for c in channels:
    lfp = pd.Series(dtype = 'object')
    print(runIntervals.as_units('s'))
    for run in runIntervals.as_units('s').iloc:
        lfp_ = bk.load.lfp(run['start'],run['end'],166,c)
        lfp = lfp.append(lfp_)

    lfp = nts.Tsd(lfp[~lfp.index.duplicated(keep='first')])
    lfps.update({c:lfp})
pos = bk.load.pos()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
     start      end
0      0.0    206.3
1   9405.5  12649.0
2  22415.0  22680.0
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
     start      end
0      0.0    206.3
1   9405.5  12649.0
2  22415.0  22680.0
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp


In [64]:
for lfp in lfps:
    filt = bk.signal.passband(lfps[lfp],5,10)
    xa = scipy.signal.hilbert(filt)
    xa = nts.Tsd(np.array(filt.index),xa)
#     x_power = scipy.stats.zscore(np.abs(xa.values)**2)
    x_power = (np.abs(xa.values)**2)
    g = scipy.signal.gaussian(1250,512)
    x_power = np.convolve(x_power,g,'same')
    x_power = nts.Tsd(xa.times(),x_power)
    f,t,Sxx = scipy.signal.spectrogram(lfps[lfp].as_units('s'),fs=1250,mode= 'psd',nperseg=2500,noverlap=1250)
    
    plt.figure()
    s = plt.subplot(3,1,1)
    plt.plot(lfps[lfp].as_units('s'))
    plt.plot(filt.as_units('s'))
    plt.subplot(3,1,2,sharex=s)
    plt.plot(x_power.as_units('s'))
    plt.subplot(3,1,3,sharex=s)
    plt.pcolormesh(t, f, Sxx)

    

In [50]:
np.mean(x_power.as_units('s')[86:89])

22655.308116542023

In [51]:
np.mean(x_power.as_units('s')[92:94])

8792.089292992887

In [67]:
plt.clim(0,25000)
plt.ylim(0,20)

(0.0, 20.0)

In [9]:
bk.load.current_session(path)
states = bk.load.states()


stru = [['HPC',23],['BLA',105]]
for i in stru:
    #lfp
    lfp = bk.load.lfp(states['sws'].as_units('s')['start'][0],states['sws'].as_units('s')['end'][0]+50,166,i[1])
    filt = bk.signal.passband(lfp,50,80)
    
    
    f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=2500,noverlap=1250)
    t = t+states['sws'].as_units('s')['start'][0]
    
    widths = np.arange(1, 100)
    cwtmatr = scipy.signal.cwt(lfp.as_units('s'),scipy.signal.morlet,widths)
    
    xa = scipy.signal.hilbert(filt)
    power = scipy.stats.zscore(np.abs(xa)**2)
    # g = scipy.signal.gaussian(12500,1250)

    # power = scipy.stats.zscore(np.convolve(power,g,'same'))
    power = nts.Tsd(filt.times(),power)

    plt.figure()
    sub = plt.subplot(4,1,1)
    plt.title('LFP and filter')
    plt.plot(lfp.as_units('s'))
    plt.plot(filt.as_units('s'))
    bk.plot.intervals(states['Rem'].iloc[0],col = 'orange')


    plt.subplot(4,1,2,sharex = sub)
    plt.pcolormesh(t, f, Sxx)
    plt.clim(0,20000)
    plt.ylim(0,80)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    

    
    plt.subplot(4,1,3,sharex=sub)
    plt.title('Wavelet')
    t = np.linspace(t[0],t[-1],490000)
    plt.pcolormesh(t,widths,cwtmatr)
    plt.clim(0,10000)


    plt.subplot(4,1,4,sharex=sub)
    plt.plot(power.as_units('s'))
    plt.title("Power Hilbert")

    plt.show()
    plt.suptitle(i[0])

Rat : 8 on day : 7
Working with session Rat08-20130715 @ Z:\Rat08\Rat08-20130715
Load LFP from Rat08-20130715.lfp
<class 'pandas.core.series.Series'>
Load LFP from Rat08-20130715.lfp
<class 'pandas.core.series.Series'>


Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_qt5.py", line 496, in _draw_idle
    self.draw()
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py", line 393, in draw
    self.figure.draw(self.renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py", line 1736, in draw
    renderer, self, artists, self.suppressComposite)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\image.py", line 137, in _draw_list_compositing_images
    a.draw(renderer)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\artist.py", line 38, in draw_wrapper
    return draw(artist, renderer, *args, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\axes\_base.py", line 2630, in draw
    mimage._dra

In [46]:
print(np.shape(t),np.shape(widths),np.shape(cwtmatr))
print(np.shape(t),np.shape(f),np.shape(Sxx))

(391,) (99,) (99, 490000)
(391,) (1251,) (1251, 391)


In [47]:
f

array([0.000e+00, 5.000e-01, 1.000e+00, ..., 6.240e+02, 6.245e+02,
       6.250e+02])

In [9]:
filt = bk.signal.passband(lfp,4,8)
xa = scipy.signal.hilbert(filt.values)
x_phase = np.angle(xa)
x_phase = nts.Tsd(np.array(lfp.index),x_phase)

plt.figure()
sub = plt.subplot(2,1,1)
plt.plot(filt)
plt.subplot(2,1,2,sharex = sub)
plt.plot(x_phase,'.k')

In [54]:
lfp = bk.load.lfp(0,20000,166,13)

Load LFP from Rat08-20130713.lfp


In [55]:
lfp_rem = lfp.restrict(states['Rem'])
f,Pxx = scipy.signal.welch(lfp_rem.values,fs=1250,nperseg = 2500,noverlap = 1250)
# plt.figure()
plt.plot(f,Pxx)

In [ ]:
plt

In [179]:
del states['drowsy']

lfps = {}
for lap in laps:
    lfp_ = lfp.restrict(laps[lap])
    lfps.update({lap:lfp_})

for state in states:
    lfp_ = lfp.restrict(states[state])
    lfps.update({state:lfp_})

In [7]:

# x_phase = np.angle(xa.values)
# x_phase = nts.Tsd(np.array(filtered.index),x_phase)

# x_rem = x_power.restrict(states['Rem'])
# x_sws = x_power.restrict(states['sws'])

# f_rem = scipy.signal.savgol_filter(x_rem.values,3001,2)
# f_sws = scipy.signal.savgol_filter(x_sws.values,3001,2)


# plt.plot(x_rem.as_units('s'))
# plt.plot(x_sws.as_units('s'))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [286]:

# plt.plot(x_power.restrict(laps['run']))
# plt.plot(pos.restrict(laps['run']))
# plt.plot(position_power_run)

In [40]:
p = ['run','safe','danger']

for l in p:
    print(l)
    f, Pxx_den = scipy.signal.welch(lfps[l], 1250,nperseg=1024,nfft=1250*10,scaling='density')
    plt.plot(f, Pxx_den)
    plt.xlim(0,100)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.show()
plt.xlim(2,20)
plt.legend([i for i in p])

run
safe
danger


In [353]:
runInt

,start,end
0,206,9405


In [68]:
plt.plot(position_power.as_units('s'))
bk.plot.intervals(laps['run'],'black')

In [17]:
path = bk.load.sessions()['Path'][5]
bk.load.current_session(path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

In [18]:
lfp = bk.load.loadLFP(bk.load.path+ '\\' + bk.load.session+".lfp",166,101)

opening


In [287]:
states = bk.load.states(bk.load.session)

In [288]:
sub = plt.subplot(3,1,1)
plt.plot(lfp.as_units('s')[3200:3400])
plt.plot(filtered.as_units('s')[3200:3400])
bk.plot.intervals(states['Rem'],'orange')
bk.plot.intervals(states['sws'],'grey')
plt.subplot(3,1,2,sharex = sub)
plt.plot(x_power.as_units('s')[3200:3400])
plt.subplot(3,1,3,sharex = sub)
plt.plot(x_phase.as_units('s')[3200:3400])
# bk.plot.intervals(states['Rem'],'orange',time_units='us')
# bk.plot.intervals(states['sws'],'grey',time_units='us')

In [16]:
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=2500,noverlap=1250)
mask = (3000<t) & (t<3500)
plt.pcolormesh(t[mask], f, Sxx[:,mask])
plt.clim(0,20000)
plt.ylim(0,80)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

NameError: name 'lfp' is not defined

In [32]:
lfp_rem = lfp.restrict(states['Rem'])

In [355]:
# f_rem = np.convolve(x_rem.values,k,'same')
# f_sws = np.convolve(x_sws.values,k,'same')


In [33]:
sub = plt.subplot(3,1,1)
plt.plot(lfp_rem)
f = bk.signal.passband(lfp_rem,5,10)
plt.plot(bk.signal.passband(lfp_rem,5,10))
plt.subplot(3,1,2 ,sharex = sub)
xa = scipy.signal.hilbert(f)
xa = nts.Tsd(np.array(f.index),xa)
x_phase = np.angle(xa.values)
x_phase = nts.Tsd(np.array(f.index),x_phase)

x_power = 
plt.plot(x_phase)

In [10]:
import pywt

In [11]:
import scaleogram

ModuleNotFoundError: No module named 'scaleogram'

In [23]:
scales = np.arange(1,40)
coef,freq = pywt.cwt(lfp,scales,'gaus1')

In [26]:
plt.pcolormesh(t,freq,coef)